## Example ADVANCED mode recipe - normalization layer extra parameters tuning by SLaNC


In [1]:
import warnings
warnings.filterwarnings("ignore")


1. Instantiate a `torch` model from source, HF hub in this case.

In [2]:
from transformers import pipeline

model = "openai/clip-vit-base-patch32"
task = "zero-shot-image-classification"

task_cases = [
    dict(
        images="http://images.cocodataset.org/val2017/000000039769.jpg",
        candidate_labels=[
            "a photo of cats",
            "a photo of dogs",
        ],
    ),
    dict(
        images="http://images.cocodataset.org/val2017/000000397133.jpg",
        candidate_labels=[
            "a kitchen scene",
            "a living room scene",
        ],
    ),
]

pipe = pipeline(
    task=task,
    model=model,
    device_map="auto",
)

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


[[{'score': 0.9988455772399902, 'label': 'a photo of cats'},
  {'score': 0.0011544461594894528, 'label': 'a photo of dogs'}],
 [{'score': 0.9962840676307678, 'label': 'a kitchen scene'},
  {'score': 0.0037159069906920195, 'label': 'a living room scene'}]]

2. Transform into `DmxModel`; this does not change the functional behavior.

In [3]:
from dmx.compressor import DmxModel

pipe.model = DmxModel.from_torch(pipe.model)

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

[[{'score': 0.9988455772399902, 'label': 'a photo of cats'},
  {'score': 0.0011544550070539117, 'label': 'a photo of dogs'}],
 [{'score': 0.9962841868400574, 'label': 'a kitchen scene'},
  {'score': 0.0037158718332648277, 'label': 'a living room scene'}]]

3. Configure to BASIC mode; this should bring in all VSIMD approximations with default config.

In [4]:
pipe.model.to_basic_mode()

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

[[{'score': 0.9199306964874268, 'label': 'a photo of cats'},
  {'score': 0.08006926625967026, 'label': 'a photo of dogs'}],
 [{'score': 0.9805154800415039, 'label': 'a kitchen scene'},
  {'score': 0.019484540447592735, 'label': 'a living room scene'}]]

4. SLaNC calibrate `LayerNorm` instances.

In [5]:
from dmx.compressor import nn
import re
from dmx.compressor.advanced_recipe import (
    DmxSLaNCHyperparams,
    DmxSLaNCRecipe,
)


def hp_gen(_model) -> dict:
    _hp = {}    
    for _n, _m in _model.named_dmx_modules():
        if isinstance(_m, nn.LayerNorm):
            if "layer_norm1" in _n and ".0." not in _n:
                layer_num = int(''.join(re.findall(r'\d', _n)))
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_mlp",
                    mlp_type="standard",
                    device=_m.weight.device,
                    prev_ln_weight=pipe.model.get_submodule(
                        _n.replace("layer_norm1", "layer_norm2", -1)
                        .replace(str(layer_num), str(layer_num - 1), -1)),
                    fc1=pipe.model.get_submodule(
                        _n.replace("layer_norm1", "mlp.fc1", -1)
                        .replace(str(layer_num), str(layer_num - 1), -1)),
                    fc2=pipe.model.get_submodule(
                        _n.replace("layer_norm1", "mlp.fc2", -1)
                        .replace(str(layer_num), str(layer_num - 1), -1)),
                )
            elif "layer_norm2" in _n:
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_attn",
                    device=_m.weight.device,
                    prev_ln_weight=pipe.model.get_submodule(_n.replace("layer_norm2", "layer_norm1", -1)),
                    v_proj=pipe.model.get_submodule(_n.replace("layer_norm2", "self_attn.v_proj", -1)),
                    o_proj=pipe.model.get_submodule(_n.replace("layer_norm2", "self_attn.out_proj", -1)),
                )
            elif "final_layer_norm" in _n:
                layers = pipe.model.get_submodule(_n.replace("final_layer_norm", "encoder.layers", -1))
                layers = list(layers.children())
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_mlp",
                    mlp_type="standard",
                    device=_m.weight.device,
                    prev_ln_weight=layers[-1].layer_norm2,
                    fc1=layers[-1].mlp.fc1,
                    fc2=layers[-1].mlp.fc2
                )
            elif "post_layernorm" in _n:
                layers = pipe.model.get_submodule(_n.replace("post_layernorm", "encoder.layers", -1))
                layers = list(layers.children())
                _hp[_m] = DmxSLaNCHyperparams(
                    position="post_mlp",
                    mlp_type="standard",
                    device=_m.weight.device,
                    prev_ln_weight=layers[-1].layer_norm2,
                    fc1=layers[-1].mlp.fc1,
                    fc2=layers[-1].mlp.fc2
                )

            elif "pre_layrnorm" in _n or (".0." in _n and "layer_norm1") in _n:
                _hp[_m] = DmxSLaNCHyperparams(
                    position="first",
                    device=_m.weight.device,   
                )
    return _hp

with DmxSLaNCRecipe(hp_gen).applied_to(pipe.model):
    print("SLaNC done!")

# -------------------------------------------------------------------------------
[pipe(**_tc) for _tc in task_cases]

SLaNC done!


[[{'score': 0.998742401599884, 'label': 'a photo of cats'},
  {'score': 0.001257657422684133, 'label': 'a photo of dogs'}],
 [{'score': 0.9972234964370728, 'label': 'a kitchen scene'},
  {'score': 0.0027764851693063974, 'label': 'a living room scene'}]]